## Save and Restore
- Save the paramters of a model to disk
- Restore the paramters of a model from the checkpoint
- Customized model initialization
- How to finetune a model



### Save the paramters of your model to disk.

In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
tf.reset_default_graph()

/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
# model define
def prelu(x, name = 'prelu'):
    with tf.variable_scope(name):
        alphas = tf.get_variable('alpha', x.get_shape()[-1], initializer=tf.constant_initializer(0.25), 
                                 regularizer = l2_regularizer, dtype = tf.float32)
    pos = tf.nn.relu(x)
    neg = tf.multiply(alphas,(x - abs(x)) * 0.5)
    return pos + neg

def first_conv(input, num_output, name):
    with tf.variable_scope(name):
        zero_init = tf.zeros_initializer()
        network = tf.layers.conv2d(input, num_output, kernel_size = [3, 3], strides = (2, 2), 
                                   padding = 'same', kernel_initializer = xavier, bias_initializer = zero_init, 
                                   kernel_regularizer = l2_regularizer, bias_regularizer = l2_regularizer)
        network = prelu(network, name = name)
        return network

def block(input, name, num_output):
    with tf.variable_scope(name):
        network = tf.layers.conv2d(input, num_output, kernel_size = [3, 3], strides = [1, 1], padding = 'same', 
                                   kernel_initializer = tf.random_normal_initializer(stddev=0.01), 
                                   use_bias = False , kernel_regularizer = l2_regularizer)
        network = prelu(network, name = 'name'+ '1')
        network = tf.layers.conv2d(network, num_output, kernel_size = [3, 3], strides = [1, 1], padding = 'same', 
                                   kernel_initializer = tf.random_normal_initializer(stddev=0.01), 
                                   use_bias = False, kernel_regularizer = l2_regularizer)
        network = prelu(network, name = 'name'+ '2')
        network = tf.add(input, network)
        return network
l2_regularizer= tf.contrib.layers.l2_regularizer(1.0)
xavier = tf.contrib.layers.xavier_initializer_conv2d() 

def get_shape(tensor):
    static_shape = tensor.shape.as_list()
    dynamic_shape = tf.unstack(tf.shape(tensor))
    dims = [s[1] if s[0] is None else s[0] for s in zip(static_shape,dynamic_shape)]
    return dims

def infer(input,embedding_size=512):
    with tf.variable_scope('conv1_'):
        network = first_conv(input, 64, name = 'conv1')
        network = block(network, 'conv1_23', 64)
    with tf.variable_scope('conv2_'):
        network = first_conv(network, 128, name = 'conv2')
        network = block(network, 'conv2_23', 128)
        network = block(network, 'conv2_45', 128)
    with tf.variable_scope('conv3_'):
        network = first_conv(network, 256, name = 'conv3')
        network = block(network, 'conv3_23', 256)
        network = block(network, 'conv3_45', 256)
        network = block(network, 'conv3_67', 256)
        network = block(network, 'conv3_89', 256)
    with tf.variable_scope('conv4_'):
        network = first_conv(network, 512, name = 'conv4')
        network = block(network, 'conv4_23', 512)
    with tf.variable_scope('feature'):
        #BATCH_SIZE = network.get_shape()[0]
        dims = get_shape(network) # [1, 7, 6, 512]
        #print(dims)
        #BATCH_SIZE = tf.shape(network)[0]
        #feature = tf.layers.dense(tf.reshape(network,[BATCH_SIZE, -1]), 512, kernel_regularizer = l2_regularizer, 
        #                          kernel_initializer = xavier)
        feature = tf.layers.dense(inputs=tf.reshape(network,[dims[0], np.prod(dims[1:])]), units=embedding_size, 
                                  kernel_regularizer = l2_regularizer, kernel_initializer = xavier)
    return feature

In [25]:
# inference
tf.reset_default_graph() # reset the default graph before constructing graph
#image = tf.random_normal([1,112,96,3])
#image = tf.constant(np.random.normal(size=[1,112,96,3]),dtype=tf.float32)
img_init = np.random.normal(size=[1,112,96,3]) # create a random image as input
image = tf.constant(img_init,dtype=tf.float32) #放入tf.constant
with tf.variable_scope('sphere_net'):
    feature = infer(image)

In [26]:
vars = tf.global_variables() # 獲取全局變量
print(vars[0])

<tf.Variable 'sphere_net/conv1_/conv1/conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>


In [27]:
# loss
pred = tf.layers.dense(feature,1,name='pred')
#print(pred.get_shape())
loss = tf.nn.l2_loss(pred-1)
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [30]:
# train and save model
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        loss_np,pred_np,_ = sess.run([loss,pred,optimizer])
        if i % 100 ==0:
            print(loss_np,pred_np)
            # save params to disk
            saver.save(sess,'model/sphere',global_step=i,write_meta_graph=False)
            pred_nd = sess.run(pred)
            #print(pred_nd, "\n")
      

0.5566992 [[-0.05517695]]
0.013193842 [[0.83755714]]
0.0002495511 [[0.9776594]]
4.5427214e-06 [[0.9969858]]
1.3147103e-07 [[0.9994872]]


In [99]:
# restore the model and test
tf.reset_default_graph()
ckpt = tf.train.get_checkpoint_state('model') # 指定模型所在的目錄路徑"model", read from directory "model"

print(type(ckpt))
print(ckpt.model_checkpoint_path)
image = tf.constant(img_init,dtype=tf.float32)
print(image)
#image = tf.random_normal([1,112,96,3])
#image = tf.constant(np.random.normal(size=[1,112,96,3]),dtype=tf.float32)
with tf.variable_scope('sphere_net'):
    feature = infer(image)
pred = tf.layers.dense(feature,1,name='pred')
saver = tf.train.Saver(tf.trainable_variables()) # 指定trainable_variables才恢復

with tf.Session() as sess:
    saver.restore(sess,ckpt.model_checkpoint_path) # 指定從哪個checkpoint file恢復
    #saver.restore(sess,'model/sphere-100')
    print(sess.run(pred)) # 使用讀取的weight，執行prediction

<class 'tensorflow.python.training.checkpoint_state_pb2.CheckpointState'>
model/sphere-400
Tensor("Const:0", shape=(1, 112, 96, 3), dtype=float32)
[1, 7, 6, 512]
INFO:tensorflow:Restoring parameters from model/sphere-400
[[ 0.99942195]]


## Customized model initialization

In [35]:
tf.reset_default_graph()
from tensorflow.python.training import checkpoint_utils
image = tf.constant(img_init,dtype=tf.float32)
with tf.variable_scope('sphere_net_left'):
    feature_left = infer(image)
with tf.variable_scope('sphere_net_right'):
    feature_right = infer(image)    
#vars = tf.trainable_variables()
#print(len(vars))
#print(vars[1])
pred = tf.layers.dense(feature_left,1,name='pred')
dense_var = [var for var in tf.trainable_variables() if 'pred' in var.name]
print(dense_var)
saver = tf.train.Saver(tf.trainable_variables())
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_left/'})
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_right/'})
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'pred/':'pred/'})

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    #saver.restore(sess,ckpt.model_checkpoint_path)
    #saver.restore(sess,'model/sphere-100')
    print(sess.run(pred))


92


"\npred = tf.layers.dense(feature_left,1,name='pred')\ndense_var = [var for var in tf.trainable_variables() if 'pred' in var.name]\nprint(dense_var)\nsaver = tf.train.Saver(tf.trainable_variables())\ncheckpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_left/'})\ncheckpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_right/'})\ncheckpoint_utils.init_from_checkpoint('model/sphere-100',{'pred/':'pred/'})\n\nwith tf.Session() as sess:\n    tf.global_variables_initializer().run(session=sess)\n    #saver.restore(sess,ckpt.model_checkpoint_path)\n    #saver.restore(sess,'model/sphere-100')\n    print(sess.run(pred))\n"

## Finetune your model

In [45]:
tf.reset_default_graph()
from tensorflow.python.training import checkpoint_utils
image = tf.constant(img_init,dtype=tf.float32)
with tf.variable_scope('sphere_net'):
    feature_left = infer(image)
#with tf.variable_scope('sphere_net_right'):
#    feature_right = infer(image)
pred = tf.layers.dense(feature_left,1,name='pred_new') # 若改了新的名字，當發現與之前訓練好的名字不同時，會報錯
loss = tf.nn.l2_loss(pred-1)
#optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.GradientDescentOptimizer(0.001)
dense_var = [var for var in tf.trainable_variables() if 'pred_new' in var.name]
grads = optimizer.compute_gradients(loss,dense_var)
train_op = optimizer.apply_gradients(grads)

print(dense_var)
saver_vars = [var for var in tf.trainable_variables() if 'pred_new' not in var.name]
saver = tf.train.Saver(saver_vars) # 指定saver_vars變量進行恢復
#saver = tf.train.Saver(tf.trainable_variables()) # 指定對所有可訓練(trainable) 變量進行恢復
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_left/'})
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_right/'})
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'pred/':'pred/'})

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    #saver.restore(sess,ckpt.model_checkpoint_path)
    saver.restore(sess,'model/sphere-100')
    for i in range(500):
        loss_np,pred_np,_ = sess.run([loss,pred,train_op])
        if i % 100 ==0:
            print(loss_np,pred_np)
            #saver.save(sess,'model/sphere',global_step=i,write_meta_graph=False)
            pred_nd = sess.run(pred)
            print(pred_nd)
      

[<tf.Variable 'pred_new/kernel:0' shape=(512, 1) dtype=float32_ref>, <tf.Variable 'pred_new/bias:0' shape=(1,) dtype=float32_ref>]
INFO:tensorflow:Restoring parameters from model/sphere-100
0.40562248 [[0.09930862]]
[[0.10302769]]
0.17730841 [[0.40450284]]
[[0.4069617]]
0.07750623 [[0.6062838]]
[[0.60790944]]
0.03388004 [[0.73969233]]
[[0.7407671]]
0.014809893 [[0.827896]]
[[0.8286067]]
